In [4]:
from swift import Swift
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['MODELSCOPE_API_TOKEN'] = 'b07e1120-5a1f-4044-84cb-1cb3c2e415fb'

from swift.llm import (
    get_model_tokenizer, get_template, inference,
    get_default_template_type, inference_stream
)
from swift.utils import seed_everything

model_choices = [
    ("deepseek-vl-1_3b-chat", "/home/rifo.genadi/material_classification/output/deepseek-vl-1_3b-chat/v0-20241115-100136/checkpoint-422"),
    ("internvl2-1b", "/home/rifo.genadi/material_classification/output/internvl2-1b/v0-20241115-213022/checkpoint-422"),
    ("deepseek-janus-1_3b", "/home/rifo.genadi/material_classification/output/deepseek-janus-1_3b/v0-20241115-100111/checkpoint-422")
    ]

choice = 1
use_trained = True

model_type, model_checkpoint = model_choices[choice]
template_type = get_default_template_type(model_type)
print(f'template_type: {template_type}')
model, tokenizer = get_model_tokenizer(model_type, model_kwargs={'device_map': "cuda:0"})
if use_trained:
    model = Swift.from_pretrained(model, model_checkpoint)


model.generation_config.max_new_tokens = 128
model.generation_config.temperature=0.01
template = get_template(template_type, tokenizer)
seed_everything(42)

[INFO:swift] Downloading the model from ModelScope Hub, model_id: OpenGVLab/InternVL2-1B


template_type: internvl2


[WARNING:modelscope] Using branch: master as version is unstable, use with caution
[INFO:swift] Loading the model using model_dir: /home/rifo.genadi/.cache/modelscope/hub/OpenGVLab/InternVL2-1B
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}
[INFO:swift] model.max_model_len: 32768
[INFO:swift] Global seed set to 42


42

In [5]:
from datasets import load_dataset

ds = load_dataset("Erland/AI701_project")
test_images = ds['test']['resized_image']
y_true = ds['test']['label']
id2label = {
    0: 'ceramic',
    1: 'fabric',
    2: 'food',
    3: 'glass',
    4: 'metal',
    5: 'other',
    6: 'paper',
    7: 'plastic',
    8: 'unknown',
    9: 'wood'
}
y_true = [id2label[y] for y in y_true]

model.generation_config.max_new_tokens = 144
model.generation_config.temperature=0.01
template = get_template(template_type, tokenizer)

# query = f'<image>\nWhat material is this object made of? Respond unknown if you are not sure. Answer only with the name of the material.\nShort answer:'
query = f"You are a helpful language and vision assistant. You are able to understand the visual content that the user provides, and assist the user with a variety of tasks using natural language.\n\nUser: <image_placeholder>\nWhat material is this object made of? Respond unknown if you are not sure. Answer only with the name of the material.\nShort answer:\n\nAssistant:"

responses = []
from tqdm.auto import tqdm
for i in tqdm(range(len(y_true))):
    images = [test_images[i]]
    response, history = inference(model, template, query, images=images)
    responses.append(response)
# print(f'query: {query}')
# print(f'response: {response}')

correct = 0
for y_pred, y in zip(responses, y_true):
    if y_pred.lower().strip() == y.lower().strip():
        correct += 1

print(correct/len(y_true)*100, "%")

  0%|          | 0/1115 [00:00<?, ?it/s]

72.46636771300449 %
